In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# Fetch the 20newsgroups dataset
data = fetch_20newsgroups(subset='all', shuffle=True, random_state=42)

In [ ]:
# Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    data.data, data.target, test_size=0.2, random_state=42
)

In [ ]:
# Convert text data to numerical vectors using TF-IDF
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train).toarray()
X_test = vectorizer.transform(X_test).toarray()

# Convert numpy arrays to PyTorch tensors
X_train = torch.from_numpy(X_train).float()
X_test = torch.from_numpy(X_test).float()
y_train = torch.from_numpy(y_train).long()
y_test = torch.from_numpy(y_test).long()

In [ ]:
# Define the CNN model
class CNN(nn.Module):
    def __init__(self, num_classes):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv1d(1, 256, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(256, 256, kernel_size=3, padding=1)
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)
        self.fc = nn.Linear(256 * 125, num_classes)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

# Initialize the model
model = CNN(num_classes=len(data.target_names))

In [ ]:
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Using PyTorch Accelaration using Apple Silicon 'MPS' GPU
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
model.to(device)

In [ ]:
# Training loop
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in tqdm(zip(X_train, y_train), desc=f"Epoch {epoch+1}/{num_epochs}", total=len(y_train)):
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        inputs = inputs.unsqueeze(0)  # Add batch dimension
        outputs = model(inputs)
        loss = criterion(outputs, labels.unsqueeze(0))
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1} loss: {running_loss / len(y_train)}")

In [ ]:
# Evaluation loop
model.eval()
predicted_labels = []
actual_labels = []

for inputs, labels in tqdm(test_dataloader, desc="Evaluation"):
    inputs = inputs.to(device)
    labels = labels.to(device)

    with torch.no_grad():
        outputs = model(inputs)

    _, predicted = torch.max(outputs.data, 1)
    predicted_labels.extend(predicted.cpu().numpy())
    actual_labels.extend(labels.cpu().numpy())

In [ ]:
# Print actual vs predicted labels
print("Actual vs Predicted Labels:")
for actual, predicted in zip(actual_labels, predicted_labels):
    print(f"Actual: {data.target_names[actual]} | Predicted: {data.target_names[predicted]}")


In [ ]:
# Classification report
print("\nClassification Report:")
print(classification_report(actual_labels, predicted_labels, target_names=data.target_names))

In [ ]:
# Confusion matrix
confusion_matrix = torch.zeros(len(data.target_names), len(data.target_names))

for actual, predicted in zip(actual_labels, predicted_labels):
    confusion_matrix[actual][predicted] += 1

plt.figure(figsize=(10, 8))
sns.heatmap(confusion_matrix, annot=True, fmt="g", xticklabels=data.target_names, yticklabels=data.target_names)
plt.xlabel("Predicted Labels")
plt.ylabel("Actual Labels")
plt.title("Confusion Matrix")
plt.show()